In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import csv
import time
from datetime import datetime

# ..................................................................................................
# --- PASSAR    P A R A M E T R O S    F I X O S .............
# ..................................................................................................

str_nome = "Marcos Heil Costa"
str_nome = "\"" + str_nome + "\""
str_nome_abreviado = "MHCOS"

# Início do ANO, dia e mês
str_mes_ini = "01/01"

# Fim do ANO, dia e mês
str_mes_fim = "31/12"

# ANO inicial
ano_ini = 1990

dia_atual = datetime.now()

# Nome do ARQUIVO que ARMAZENARÁ a LISTA com  TODOS OS JORNAIS da Consulta atual
nm_arquivo_resultado = str_nome_abreviado + "_"  + str(ano_ini) + ".csv"

# Path Do Navegador
chrome = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

###driver = webdriver.Chrome(executable_path=r'/Users/nayaramcd/Documents/UFSCar/Pesquisa/Dados/pesquisa_dou_python/chromedriver')

# ..................................................................................................
# --- Iniciar o browser
# ..................................................................................................
###chrome = webdriver.Chrome(driver)

## chrome.maximize_window()
## chrome.get("https://pesquisa.in.gov.br/")


# ..................................................................................................
# --- EXECUTAR  a n o   a    a n o
# ..................................................................................................
while ano_ini <= 2021:
    chrome.maximize_window()
    chrome.get("https://pesquisa.in.gov.br/")
    time.sleep(5)
    elem = chrome.find_element_by_id("txtPesquisa_avancada").send_keys(str_nome) 
    element = chrome.find_element_by_xpath("//input[@id='dt_inicio_avancada']")
    element.clear()
    element.send_keys(str_mes_ini)   
    
    element_dt_fim = chrome.find_element_by_xpath("//input[@id='dt_fim_avancada']")
    element_dt_fim.clear()
    element_dt_fim.send_keys(str_mes_fim)   

    delay = 2 # seconds
    element_ano = WebDriverWait(chrome, delay).until(EC.presence_of_element_located((By.ID, 'ano_avancada')))
    element_ano.send_keys(ano_ini)   

    search_button = chrome.find_element_by_id("pesquisa02_0")
    search_button.click()

# Guardar o html
    chrome.page_source
    doc = BeautifulSoup(chrome.page_source, "html.parser")
   
        
#..................................................................................................
# --- CALCULAR SE ACHOU REGISTRO, QUANTOS E O TOTAL DE PAGINAS A PESQUISAR.
# ..................................................................................................
    delay = 1 # seconds
    # se encontrou resultados, tem a nav com id = navigation
    element_paginaco = WebDriverWait(chrome, delay).until(EC.presence_of_element_located((By.ID, 'navigation'))).text    
       
    #print("element_paginaco", element_paginaco, len(element_paginaco))  
    tam_str = element_paginaco.index(" ")
    #print(f" tam_str é: {tam_str}")
    nro = element_paginaco[0:tam_str]
    
    if nro.isnumeric():
        tot_reg = int(element_paginaco[0:tam_str] )
        tot_pag = int(element_paginaco[0:tam_str] ) // 10  + 1
    else:
        #print("Nao achou resultado")
        tot_reg = 0
        tot_pag = 1
       
    #print(f"{tot_reg}")
    
    with open(nm_arquivo_resultado, 'a', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        #writer.writerow(doc)
        for link in doc.find_all('a',attrs={'class':'titulo_jornal'}):
            titulo = link.string
            caminho = link.get('href')
            writer.writerow([dia_atual.strftime('%Y%m%d %H:%M'), nm_arquivo_resultado, str_nome, str_nome_abreviado, ano_ini, tot_pag, tot_reg, titulo, caminho])
            
        
        # "SE NAO ENCONTRAR RESULTADOS, ESCREVE UMA LINHA NO CSV PRA INFORMAR QUE TOTAL DE REGISTROS = 0"
        if tot_reg == 0:
            writer.writerow([dia_atual.strftime('%Y%m%d %H:%M'), nm_arquivo_resultado, str_nome, str_nome_abreviado, ano_ini, tot_pag, tot_reg, "", ""])
           
        # "SE TEM PAGINAÇAO, PRECISA PERCORRER A PROXIMA PAGINA E SALVAR OS LINKS"    
            # Paginação!
        
            for pag in range(tot_pag):
                l=chrome.find_element_by_link_text("Próximo")
                l.click()

                doc2 = BeautifulSoup(chrome.page_source, "html.parser")

                # "JA CLICOU NA PROXIMA PAGINA, GUARDOU CONTEUDO EM DOC"
                for link2 in doc2.find_all('a',attrs={'class':'titulo_jornal'}):
                    titulo_pag = link2.string
                    caminho_pag = link2.get('href')
                    writer.writerow([dia_atual.strftime('%Y%m%d %H:%M'), nm_arquivo_resultado, str_nome, str_nome_abreviado, ano_ini, tot_pag, tot_reg, titulo_pag, caminho_pag])

    time.sleep(5)    
    ano_ini += 1



====== WebDriver manager ======
Current google-chrome version is 109.0.5414
Get LATEST chromedriver version for 109.0.5414 google-chrome
There is no [mac64] chromedriver for browser 109.0.5414 in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/109.0.5414.74/chromedriver_mac64.zip
Driver has been saved in cache [/Users/nayaramcd/.wdm/drivers/chromedriver/mac64/109.0.5414.74]
/var/folders/c4/60twv_q571xgrgqwbsdrgc1m0000gn/T/ipykernel_49057/1425992084.py:56: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = chrome.find_element_by_id("txtPesquisa_avancada").send_keys(str_nome)
/var/folders/c4/60twv_q571xgrgqwbsdrgc1m0000gn/T/ipykernel_49057/1425992084.py:57: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element = chrome.find_element_by_xpath("//input[@id='dt_inicio_avancada']")
/var/folders/c4/60twv_q571xgrgqwbsdrgc1m0000gn/T/

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000101474fa8 chromedriver + 4886440
1   chromedriver                        0x00000001013f2643 chromedriver + 4351555
2   chromedriver                        0x0000000101040b27 chromedriver + 477991
3   chromedriver                        0x000000010108576c chromedriver + 759660
4   chromedriver                        0x00000001010859c1 chromedriver + 760257
5   chromedriver                        0x00000001010c9204 chromedriver + 1036804
6   chromedriver                        0x00000001010ab5fd chromedriver + 914941
7   chromedriver                        0x00000001010c65fe chromedriver + 1025534
8   chromedriver                        0x00000001010ab3a3 chromedriver + 914339
9   chromedriver                        0x000000010107557f chromedriver + 693631
10  chromedriver                        0x0000000101076b1e chromedriver + 699166
11  chromedriver                        0x0000000101441b9e chrome